## First Aproach

- Simular $n$ termoeléctricas ($t_i$ es el valor de generación máxima de la termoeléctrica $i$)
- El día 0 funcionan todas
- Si todas funcionan Oferta $\geq$ Demanda
- Simular tiempo de rotura (Weibull)
- Todas se pueden reparar a la vez
- Tiempo de reparación Lognormal
- Existen $m$ circuitos ($c_j$ es el valor de demanda del circuito $j$)
- $\delta_{ik}$ es un parámetro binario que expresa si la termoeléctrica $i$ está activa el día $k$
- $\alpha_{jk}$ es un parámetro binario que expresa si el circuito $j$ está activo el día $k$
- Existe capacidad de almacenamiento ($a_k$ es la capacidad de almacenamiento del sistema en el día $k$)
- $a_0 = 0$
- $a_k = \sum_{i=1}^{n} t_i\delta_{ik} + a_{k-1} - \sum_{j=1}^{m} c_j\alpha_{jk}, a_k \geq 0$
- No simularemos mantenimientos planificados, solo reparación por rotura
- No tendremos en cuenta fallas internas de los circuitos
- No habrán circuitos que dependan de termoeléctricas específicas
- La afectación a los circuitos se hará de manera proporcional (con una proporción que definiremos) y rotativa, cuando sea necesario
- Al final de la simulación calcularemos una afectación (fórmula por determinar)

In [ ]:
from abc import ABC, abstractmethod

class Handler(ABC):

    @abstractmethod
    def decision():
        pass

In [2]:
import probabilistic_models as pm
import numpy as np
import random

def simulate(N, M, K, seed=0):

    if seed != 0:
        random.seed(seed)

    thermoelectrics = [random.randint(500, 1000) for _ in range(N)]
    thermoelectrics_state = [[1 for _ in range(N)]]
    circuits = [random.randint(200, 600) for _ in range(M)]
    circuits_state = [[x for x in circuits]]
    leave_service = [[] for _ in range(K)]
    re_enter_service = [[] for _ in range(K)]
    balance = 0
    demand = sum(x for x in circuits)

    for i in range(N):
        pm.break_thermoelectric(leave_service, i, 0, N)

    for t in range(1, K):
        thermoelectrics_state.append([x for x in thermoelectrics_state[t - 1]])
        
        for thermoelectric in leave_service[t]:
            thermoelectrics_state[t][thermoelectric] = 0
            pm.repair_thermoelectric(re_enter_service, thermoelectric, t, N)

        for thermoelectric in re_enter_service[t]:
            thermoelectrics_state[t][thermoelectric] = 1
            pm.break_thermoelectric(leave_service, thermoelectric, t, N)
            
        offer = sum(x * y for x, y in zip(thermoelectrics, thermoelectrics_state[t])) + balance

        while(demand > offer):
            pass

        balance = offer - demand
    